### Tests for Applying Gains Matrix

In [1]:
import cupy as cp
import numpy as np
from utils import *
from zp_puregpu_funcs_py import *
from invcov import *

hera_cal is not installed. Some gridding features unavailable.
pyfof is not installed. Some gridding features unavailable.


In [3]:
apply_gains_to_mat?
# undo_zeropad?

Signature:
apply_gains_to_mat(
    gains: cupy.ndarray,
    mat,
    edges,
    ant_1_array,
    ant_2_array,
    xp,
    is_zeropadded=True,
)
Docstring:
Apply a pair of complex gains to a matrix. Utilizes the Re/Im split.
Only accounts for "one half" of the gain application, meaning the 
function is really performing (g_1g_2*\Delta_{1,2}), where it is 
understood that antenna's 1 and 2 below to the baseline sitting at
the same row as that baseline row in the \Delta matrix. Note that although
the matrix provided may be zeropadded, the gain matrix is zeropadded here 
and as such, should always be provided in an un-zeropadded (original) format.

NOTE: Could be smart in the future to wrap the zeropadding of the gain 
    matrix in a separate function.

Params
------
gains
    1D array of Re/Im alternating gains to be applied to the source or
    diffuse matrices. Contains one set of Re/Im gains for all antennas 
    in the array. 
mat
    Gains are applied to this. Can be 2d as in origin

In [3]:
#simulation params relevant for testing application of gains to a matrix
n_ant = 15
n_bl = 2*n_ant
n_gains = 4*n_ant
n_eig = 3
xp = cp  #run things on the gpu using cupy

#this might be the easiest (and most general) way to devise an edges
# array, though we hard code an ex. edges array to be sure it fits
# the desired format of having no odd entries.
edges = (xp.unique(xp.random.randint(1, n_bl/2-1, size = 3)*2))
edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
print(f"The edges of the redundant blocks have indices{edges}")

#some random noise, diffuse, and source covariance matrices
xp = cp
sim_diff_mat = xp.random.rand(n_bl, n_eig, dtype = 'float64')
sim_gains = cp.random.rand(n_gains, dtype = 'float64') #Re/Im split + ant1 & ant 2 = 4*n_ant
ant_1_array = cp.arange(n_ant)
ant_2_array = cp.arange(n_ant, 2*n_ant)

#zeropad the noise, diff, source mats
zp_sim_diff_mat, largest_block, n_blocks = zeroPad(sim_diff_mat, edges, return_inv=False)

#Need to reshape to give an extra dimension of n_blocks to be compatible with inv cov routine
sim_diff_mat_3d = zp_sim_diff_mat.reshape(n_blocks, largest_block, n_eig)

The edges of the redundant blocks have indices[ 0  2 14 26 30]


In [4]:
applied_gains = apply_gains_to_mat(sim_gains, sim_diff_mat_3d, edges, ant_1_array, ant_1_array, cp, True)

In [5]:
print(f"Input mat has shape {sim_diff_mat_3d.shape}")
print()
print(f"Gains applied mat has shape {applied_gains.shape}")
print()
print(f"The mat with gains applied looks like:\n\n{applied_gains}")

Input mat has shape (4, 12, 3)

Gains applied mat has shape (4, 12, 3)

The mat with gains applied looks like:

[[[0.02423694 0.42415436 0.32459622]
  [0.33545082 0.03678549 0.11291223]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]]

 [[0.04085917 0.15672052 0.15294488]
  [0.06256685 0.16819805 0.10166261]
  [0.25873256 0.01474001 0.19405144]
  [0.33558665 0.51040841 0.09003162]
  [0.06645621 0.07060109 0.13992122]
  [0.04191534 0.26768344 0.13434448]
  [1.01372305 0.01275595 0.1118732 ]
  [0.35211718 0.88303851 0.58433114]
  [0.16348054 0.19547368 0.0517718 ]
  [0.15095884 0.12655385 0.07814263]
  [0.16351007 0.12131949 0.08036767]
  [0.36546468 0.40520275 0.04730554

#### Random Debugging tests

In [6]:
m = np.random.rand(2, 5, 3)
g = np.random.rand(10)

gs = g[:, None] * g[:, None]
gs = gs.reshape(m.shape[0], m.shape[1], 1)

print(gs.shape)
print(m.shape)
print(g.shape)
print(m)
print(m[:, ::2])
out = gs[:, ::2] * m[:, ::2]

(2, 5, 1)
(2, 5, 3)
(10,)
[[[0.61728211 0.976156   0.53499541]
  [0.78061534 0.12156277 0.7960254 ]
  [0.35342611 0.34604145 0.75539282]
  [0.22075206 0.46568474 0.95242102]
  [0.28729021 0.81178895 0.40647511]]

 [[0.16663594 0.2657454  0.89508378]
  [0.44143151 0.09672926 0.22475365]
  [0.17857041 0.33149725 0.51393874]
  [0.90902841 0.91434832 0.36485327]
  [0.40655417 0.11975906 0.87401215]]]
[[[0.61728211 0.976156   0.53499541]
  [0.35342611 0.34604145 0.75539282]
  [0.28729021 0.81178895 0.40647511]]

 [[0.16663594 0.2657454  0.89508378]
  [0.17857041 0.33149725 0.51393874]
  [0.40655417 0.11975906 0.87401215]]]


In [7]:
a = np.arange(5)
b = np.arange(5,10)
c = np.random.rand((a.shape[0] + b.shape[0]))
print(c)
d = c[a]
e = c[b]

print(d)
print()
print(e)
print()
print(np.concatenate((d, e)))
print()
aa = np.concatenate((d, e))

print(aa[:int(len(aa)/2)])

print(aa[int(len(aa)/2):])

[0.19007742 0.13692249 0.07096474 0.58888727 0.31496531 0.89477695
 0.83697355 0.90792351 0.87578734 0.03793271]
[0.19007742 0.13692249 0.07096474 0.58888727 0.31496531]

[0.89477695 0.83697355 0.90792351 0.87578734 0.03793271]

[0.19007742 0.13692249 0.07096474 0.58888727 0.31496531 0.89477695
 0.83697355 0.90792351 0.87578734 0.03793271]

[0.19007742 0.13692249 0.07096474 0.58888727 0.31496531]
[0.89477695 0.83697355 0.90792351 0.87578734 0.03793271]


#### Check ZP computes the same thing as corrcal

In [8]:
#Current corrcal gain application function
def apply_gains_to_mat_corrcal(gains, mat, ant_1_array, ant_2_array):
    """Apply a gain-like matrix to a provided matrix."""
    complex_gains = gains[::2] + 1j*gains[1::2]
    gain_mat = (
        complex_gains[ant_1_array,None] * complex_gains[ant_2_array,None].conj()
    )
    out = np.zeros_like(mat)
    out[::2] = gain_mat.real * mat[::2] - gain_mat.imag * mat[1::2]
    out[1::2] = gain_mat.imag * mat[::2] + gain_mat.real * mat[1::2]
    return out

In [9]:
#simulation params relevant for testing application of gains to a matrix
n_ant = 15
n_bl = 2*n_ant
n_gains = 4*n_ant
n_eig = 3
xp = cp  #run things on the gpu using cupy

#this might be the easiest (and most general) way to devise an edges
# array, though we hard code an ex. edges array to be sure it fits
# the desired format of having no odd entries.
edges = (xp.unique(xp.random.randint(1, n_bl/2-1, size = 3)*2))
edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
print(f"The edges of the redundant blocks have indices{edges}")

#some random noise, diffuse, and source covariance matrices
xp = cp
sim_diff_mat = xp.random.rand(n_bl, n_eig, dtype = 'float64')
sim_gains = cp.random.rand(n_gains, dtype = 'float64') #Re/Im split + ant1 & ant 2 = 4*n_ant
ant_1_array = cp.arange(n_ant)
ant_2_array = cp.arange(n_ant, 2*n_ant)

#zeropad the noise, diff, source mats
zp_sim_diff_mat, largest_block, n_blocks = zeroPad(sim_diff_mat, edges, return_inv=False)

#Need to reshape to give an extra dimension of n_blocks to be compatible with inv cov routine
sim_diff_mat_3d = zp_sim_diff_mat.reshape(n_blocks, largest_block, n_eig)


The edges of the redundant blocks have indices[ 0  2 16 24 30]


In [13]:
# out_corrcal = apply_gains_to_mat_corrcal(sim_gains, sim_diff_mat, ant_1_array, ant_2_array)
out_gpu = apply_gains_to_mat(sim_gains, sim_diff_mat_3d, edges, ant_1_array, ant_2_array, cp)
out_gpu_corrcal = apply_gains_to_mat(sim_gains, sim_diff_mat, edges, ant_1_array, ant_2_array, cp, False)
out_gpu_resh = undo_zeropad(out_gpu, edges, cp)

In [14]:
#should return True if my routine computes the same thing as the current corrcal routine
print(cp.allclose(out_gpu_resh, out_gpu_corrcal))

True
